In [1]:
import os
import math
import json
import gzip
import time
import calendar
from datetime import datetime
import numpy as np
import pandas as pd
from datetime import date
from multiprocessing import Pool
import warnings
warnings.filterwarnings('ignore')
SETS = './sets/' # папка с файлами с наборами

In [2]:
def default_serializer(obj): # для правильного преобразования даты в ISO формат
    if isinstance(obj, (date)):
        return obj.isoformat()
    raise TypeError(f'Object of type {obj.__class__.__name__} is not JSON serializable')
    

def load_dataset(filename): 
    if os.path.exists(SETS + filename + '.gz'):
        with gzip.open(SETS + filename + '.gz', 'rb') as gzip_ref:
            return pd.DataFrame(json.load(gzip_ref))
        
    
def save_dataset(filename):
    data = globals().get(filename)
    if data is not None:
        data = data.to_dict(orient='records')
        json_data = json.dumps(data, ensure_ascii=False, default=default_serializer)
        with gzip.open(SETS + filename + '.gz', 'wb') as gzip_file:
            gzip_file.write(json_data.encode('utf-8'))
        print(f'Сохранено {len(data)} записей в {filename}.gz')

In [4]:
pd.set_option('display.max_colwidth', None) # для отображения полного текста в ячейках
pd.set_option('display.max_columns', None) 
exams_stroi = load_dataset('exams_stroi')
exams_stroi.head()

,global_id,start,end,near_stroi,stupid
0,797946035,2017-03-01T00:00:00,2017-09-30T00:00:00,0.415290,3.800000
1,797949150,2017-03-01T00:00:00,2017-09-30T00:00:00,0.423029,3.357143
2,797949275,2017-03-01T00:00:00,2017-09-30T00:00:00,0.059235,1.100000
3,797949425,2017-03-01T00:00:00,2017-09-30T00:00:00,0.368473,6.600000
4,797949570,2017-03-01T00:00:00,2017-09-30T00:00:00,0.358154,5.000000


In [5]:
exams_stroi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5542 entries, 0 to 5541
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   global_id   5542 non-null   int64  
 1   start       5542 non-null   object 
 2   end         5542 non-null   object 
 3   near_stroi  5542 non-null   float64
 4   stupid      5542 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 216.6+ KB
